# Build the logbook for Hologram with empty or filters

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : February 02th 2022
- last update : February 02th 2022


Works directory per directory (day by day)

CCIN2P3:

- works with kernels **anaconda3_auxtel** (with libradtran) at CC
- works with kernel **python 3** locally

In [1]:
import sys
sys.path

['/Users/sylvie/MacOSX/GitHub/LSST/AuxTelComm/notebooks_usdf/runspectractor_standalone/2023_01',
 '/Users/sylvie/anaconda3/lib/python39.zip',
 '/Users/sylvie/anaconda3/lib/python3.9',
 '/Users/sylvie/anaconda3/lib/python3.9/lib-dynload',
 '',
 '/Users/sylvie/anaconda3/lib/python3.9/site-packages',
 '/Users/sylvie/anaconda3/lib/python3.9/site-packages/rubin_sim-1.0.1.dev64+g6cace9d-py3.9.egg',
 '/Users/sylvie/anaconda3/lib/python3.9/site-packages/libradtranpy-0.1.0-py3.9.egg',
 '/Users/sylvie/anaconda3/lib/python3.9/site-packages/pytest-7.2.0-py3.9.egg',
 '/Users/sylvie/anaconda3/lib/python3.9/site-packages/Django-4.1.5-py3.9.egg',
 '/Users/sylvie/anaconda3/lib/python3.9/site-packages/exceptiongroup-1.1.0-py3.9.egg',
 '/Users/sylvie/anaconda3/lib/python3.9/site-packages/iniconfig-1.1.1-py3.9.egg',
 '/Users/sylvie/anaconda3/lib/python3.9/site-packages/sqlparse-0.4.3-py3.9.egg',
 '/Users/sylvie/anaconda3/lib/python3.9/site-packages/asgiref-3.6.0-py3.9.egg',
 '/Users/sylvie/anaconda3/lib/p

In [2]:
! ls /sps/lsst/groups/auxtel/data/2023/empty~holo4_003/20221207

ls: /sps/lsst/groups/auxtel/data/2023/empty~holo4_003/20221207: No such file or directory


# Select the combination DATE, FILTER, DISPERSER

In [3]:
DATE = '20230117'
filterdispersernames = ["empty~holo4_003","BG40~holo4_003","FELH0600~holo4_003","SDSSg~holo4_003",\
                        "empty~ronchi170lpmm","BG40~ronchi170lpmm","FELH0600~ronchi170lpmm","SDSSg~ronchi170lpmm",\
                       ]
filterdispersername = filterdispersernames[0]
version="v7.0"  # new orientation

# If position need to be specified add it in this dictionnary

In [4]:
# reconstructed / good /xpos /ypos

if version == "v7.0":
    info={\

          # with red filter
          'exposure_2022031600351_postisrccd.fits':(1,1,300.,1700),\
         }
        

# Imports

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [6]:
from astropy.io import fits

In [7]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

/var/folders/m7/6pm3rhtj0c71ls7jj9mn1g740000gp/T/ipykernel_18559/3547059130.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [8]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [9]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [10]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [11]:
dir_logbooks="logbooks"

In [12]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [13]:
home=os.getenv("HOME")

In [14]:
image_path=filterdispersername
logbook_csv=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_"+ DATE +"_" + version + ".csv")
logbook_xlsx=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_" +DATE +"_" + version + ".xlsx")

# Path

In [15]:
HOSTCC=False
LAPTOP=False

In [16]:
!ls /Users/sylvie/DATA/AuxTelDATA2023/data/2023

empty~holo4_003


In [17]:
# Set path depending on which computer running (according HOSTCC)
if HOSTCC:
    path_auxtel="/sps/lsst/groups/auxtel"
    path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)

else:
    if not LAPTOP:
        path_auxtel="/Users/sylvie/DATA/AuxTelDATA2023"
        path_spectractor=os.path.join(path_auxtel,"/Users/sylvie/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)
        
    else:
        path_auxtel="/Users/dagoret/DATA/AuxTelData2023"
        path_spectractor=os.path.join(path_auxtel,"/Users/dagoret/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)
        

# Check the images exists

In [18]:
dir_images=path_images

In [19]:
all_files=os.listdir(dir_images)

In [20]:
all_files

['exposure_2023011700467_pseudo-postisrccd.fits',
 'exposure_2023011700514_pseudo-postisrccd.fits',
 'exposure_2023011700412_pseudo-postisrccd.fits',
 'exposure_2023011700561_pseudo-postisrccd.fits',
 'exposure_2023011700429_pseudo-postisrccd.fits',
 'exposure_2023011700497_pseudo-postisrccd.fits',
 'exposure_2023011700525_pseudo-postisrccd.fits',
 'exposure_2023011700647_pseudo-postisrccd.fits',
 'exposure_2023011700632_pseudo-postisrccd.fits',
 'exposure_2023011700276_pseudo-postisrccd.fits',
 'exposure_2023011700286_pseudo-postisrccd.fits',
 'exposure_2023011700334_pseudo-postisrccd.fits',
 'exposure_2023011700275_pseudo-postisrccd.fits',
 'exposure_2023011700348_pseudo-postisrccd.fits',
 'exposure_2023011700389_pseudo-postisrccd.fits',
 'exposure_2023011700285_pseudo-postisrccd.fits',
 'exposure_2023011700458_pseudo-postisrccd.fits',
 'exposure_2023011700244_pseudo-postisrccd.fits',
 'exposure_2023011700688_pseudo-postisrccd.fits',
 'exposure_2023011700562_pseudo-postisrccd.fits',


In [21]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [22]:
all_selected_files=np.array(all_selected_files)

### Sort files

In [23]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('exposure.*_(.*)_pseudo-postisrccd[.]fits$',filename)
    
    return m[0]

In [24]:
file_tag_forsorting('exposure_2022031700310_pseudo-postisrccd.fits')

'2022031700310'

In [25]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['2023011700467', '2023011700514', '2023011700412', '2023011700561',
       '2023011700429', '2023011700497', '2023011700525', '2023011700647',
       '2023011700632', '2023011700276', '2023011700286', '2023011700334',
       '2023011700275', '2023011700348', '2023011700389', '2023011700285',
       '2023011700458', '2023011700244', '2023011700688', '2023011700562',
       '2023011700411', '2023011700249', '2023011700553', '2023011700420',
       '2023011700607', '2023011700598', '2023011700678', '2023011700243',
       '2023011700599', '2023011700679', '2023011700248', '2023011700421',
       '2023011700552', '2023011700375', '2023011700349', '2023011700468',
       '2023011700239', '2023011700335', '2023011700608', '2023011700687',
       '2023011700646', '2023011700633', '2023011700590', '2023011700524',
       '2023011700457', '2023011700496', '2023011700670', '2023011700376',
       '2023011700515', '2023011700543', '2023011700430', '2023011700617',
       '2023011700298', '

In [26]:
indexes=np.argsort(all_tags)

In [27]:
sorted_files=all_selected_files[indexes]

In [28]:
sorted_files

array(['exposure_2023011700239_pseudo-postisrccd.fits',
       'exposure_2023011700243_pseudo-postisrccd.fits',
       'exposure_2023011700244_pseudo-postisrccd.fits',
       'exposure_2023011700248_pseudo-postisrccd.fits',
       'exposure_2023011700249_pseudo-postisrccd.fits',
       'exposure_2023011700265_pseudo-postisrccd.fits',
       'exposure_2023011700266_pseudo-postisrccd.fits',
       'exposure_2023011700275_pseudo-postisrccd.fits',
       'exposure_2023011700276_pseudo-postisrccd.fits',
       'exposure_2023011700285_pseudo-postisrccd.fits',
       'exposure_2023011700286_pseudo-postisrccd.fits',
       'exposure_2023011700297_pseudo-postisrccd.fits',
       'exposure_2023011700298_pseudo-postisrccd.fits',
       'exposure_2023011700316_pseudo-postisrccd.fits',
       'exposure_2023011700317_pseudo-postisrccd.fits',
       'exposure_2023011700325_pseudo-postisrccd.fits',
       'exposure_2023011700326_pseudo-postisrccd.fits',
       'exposure_2023011700334_pseudo-postisrccd

In [29]:
N=len(sorted_files)

# Build the logbook from the headers

In [30]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=hdr0['OBJECT']
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [31]:
df=pd.DataFrame()

In [32]:
len(all_airmass)

83

In [33]:
N=len(all_indexes)

## Fill with container info

In [34]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [35]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [36]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [37]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [38]:
pd.set_option('display.max_rows', None)

In [39]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2023-01-18T02:34:49.018,/Users/sylvie/DATA/AuxTelDATA2023/data/2023/em...,exposure_2023011700239_pseudo-postisrccd.fits,239,HD185975,empty_1,holo4_003,2.094525,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
1,1,2023-01-18T02:44:20.249,/Users/sylvie/DATA/AuxTelDATA2023/data/2023/em...,exposure_2023011700243_pseudo-postisrccd.fits,243,HD074000,empty_1,holo4_003,1.335267,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
2,2,2023-01-18T02:45:03.252,/Users/sylvie/DATA/AuxTelDATA2023/data/2023/em...,exposure_2023011700244_pseudo-postisrccd.fits,244,HD074000,empty_1,holo4_003,1.332128,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
3,3,2023-01-18T03:05:35.183,/Users/sylvie/DATA/AuxTelDATA2023/data/2023/em...,exposure_2023011700248_pseudo-postisrccd.fits,248,HD185975,empty_1,holo4_003,2.109638,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
4,4,2023-01-18T03:06:18.107,/Users/sylvie/DATA/AuxTelDATA2023/data/2023/em...,exposure_2023011700249_pseudo-postisrccd.fits,249,HD185975,empty_1,holo4_003,2.109952,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
5,5,2023-01-18T03:18:40.594,/Users/sylvie/DATA/AuxTelDATA2023/data/2023/em...,exposure_2023011700265_pseudo-postisrccd.fits,265,MU-COL,empty_1,holo4_003,1.010174,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
6,6,2023-01-18T03:19:23.524,/Users/sylvie/DATA/AuxTelDATA2023/data/2023/em...,exposure_2023011700266_pseudo-postisrccd.fits,266,MU-COL,empty_1,holo4_003,1.010550,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
7,7,2023-01-18T03:24:35.922,/Users/sylvie/DATA/AuxTelDATA2023/data/2023/em...,exposure_2023011700275_pseudo-postisrccd.fits,275,MU-COL,empty_1,holo4_003,1.013518,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
8,8,2023-01-18T03:25:18.849,/Users/sylvie/DATA/AuxTelDATA2023/data/2023/em...,exposure_2023011700276_pseudo-postisrccd.fits,276,MU-COL,empty_1,holo4_003,1.013958,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN
9,9,2023-01-18T03:38:30.056,/Users/sylvie/DATA/AuxTelDATA2023/data/2023/em...,exposure_2023011700285_pseudo-postisrccd.fits,285,MU-COL,empty_1,holo4_003,1.023500,30.0,...,None,None,None,NaN,NaN,NaN,NaN,,NaN,NaN


# Make logbook


In [40]:
N=len(df)
N

83

In [41]:
if N>0:
    df.to_excel(logbook_xlsx)
    df.to_csv(logbook_csv)

In [42]:
logbook_csv

'logbooks/auxtellogbook_empty~holo4_003_20230117_v7.0.csv'